# Prostate Cancer Risk Screening Using Germline Variant Data

In [1]:
!pip install scikit-allel
!pip install zarr

     |████████████████████████████████| 1.4 MB 6.4 MB/s eta 0:00:01
You should consider upgrading via the '/Users/crystalshin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 191 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 9.7 MB/s eta 0:00:01
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5035 sha256=cad9ecd67baa7eaf1946cd4a7b127b564a18cbcc6a8817d04858f7eebdd8876d
  Stored in directory: /Users/crystalshin/Library/Caches/pip/wheels/a3/d7/75/19cd0d2a893cad4bb0b2b16dd572ad2916d19c0d5ee9612511
Successfully built asciitree
You should consider upgrading via the '/Users/crystalshin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [6]:
import os

cwd = os.getcwd()
DATA = cwd + '/data'

## Identifying Pathogenic Variants

We can identify potential disease-causing variants using germline variant data. 'Clinvar' is a clinical variants database for pathogenic disease variant annotation. In this analysis, I will identify pathogenic germline variants that are likely to result in prostate cancer in 1000 genome project subjects. 1000 genome project contains variants from whole genome sequencing of 2,504 volunteers of 26 sub-population. I have downloaded the Clinvar database as a VCF file. VCF (variant call format) is the summary of all variants compared to the reference genome, and each row contains the annotation for each variant. We are using variant mapped to human reference genome version GRCh37 since the 1000 genome database is also mapped to the same version. 

In [29]:
import allel
import numpy as np
import sys

clinvar_file = DATA + "/clinvar_20220910_grch37.vcf.gz"
clinvar = allel.read_vcf(str(clinvar_file), fields="*", log=sys.stdout)

[read_vcf] 65536 rows in 0.66s; chunk in 0.66s (98744 rows/s); 1 :150469297
[read_vcf] 131072 rows in 1.31s; chunk in 0.65s (100707 rows/s); 2 :26467440
[read_vcf] 196608 rows in 1.97s; chunk in 0.66s (99585 rows/s); 2 :165946783
[read_vcf] 262144 rows in 2.66s; chunk in 0.69s (95524 rows/s); 2 :233113958
[read_vcf] 327680 rows in 3.35s; chunk in 0.69s (94696 rows/s); 3 :128627919
[read_vcf] 393216 rows in 4.03s; chunk in 0.68s (96557 rows/s); 4 :184615101
[read_vcf] 458752 rows in 4.66s; chunk in 0.63s (103692 rows/s); 5 :148377020
[read_vcf] 524288 rows in 5.33s; chunk in 0.67s (97701 rows/s); 6 :129774241
[read_vcf] 589824 rows in 5.97s; chunk in 0.64s (102314 rows/s); 7 :124532342
[read_vcf] 655360 rows in 6.66s; chunk in 0.68s (95891 rows/s); 8 :145739033
[read_vcf] 720896 rows in 7.28s; chunk in 0.62s (105669 rows/s); 9 :139418044
[read_vcf] 786432 rows in 7.88s; chunk in 0.60s (108906 rows/s); 11 :3877646
[read_vcf] 851968 rows in 8.52s; chunk in 0.65s (101525 rows/s); 11 :10821

Now, we would like to find all variants that are annotated as pathogenic. For more thorough search in real life, we would want to search all chromosomes for pathogenic variants, but for simplicity of this analysis, we will restrict the boundary to chromosome X. 

Pathogenicity can be found in column variants/CLNSIG, and the disease name could be found in variants/CLNDN. I used regular expression to find all entries containing the word "Pathogenic" in the column `variants/CLNDN`. 

In [30]:
import re
import pandas as pd

chrom_x_idx = clinvar["variants/CHROM"] == 'X'
pathogenic_idx = np.array([x == "Pathogenic" for x in clinvar["variants/CLNSIG"]])
prostate_idx = np.array(["prostate" in x.lower() for x in clinvar["variants/CLNDN"]])

idx_set = chrom_x_idx & pathogenic_idx & prca_idx

pathogenic_df = pd.DataFrame(
    dict([(k, clinvar["variants/" + k][idx_set]) 
    for k in ["CHROM", "POS", "REF", "GENEINFO", "CLNDN", "CLNSIG", "AF_TGP"]])
)
pathogenic_df["ALT"] = clinvar["variants/ALT"][idx_set, 0]

pathogenic_df

,CHROM,POS,REF,GENEINFO,CLNDN,CLNSIG,AF_TGP,ALT
0,X,66765148,T,AR:367,Malignant_tumor_of_prostate|Kennedy_disease|An...,Pathogenic,NaN,TTGC
1,X,66765196,C,AR:367|LOC109504725:109504725,Malignant_tumor_of_prostate,Pathogenic,NaN,T
2,X,66765463,G,AR:367,Malignant_tumor_of_prostate|Androgen_resistanc...,Pathogenic,NaN,A
3,X,66765802,C,AR:367,Malignant_tumor_of_prostate|not_provided,Pathogenic,NaN,T
4,X,66766013,C,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,T
5,X,66766051,G,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,C
6,X,66766163,C,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,G
7,X,66766183,T,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,C
8,X,66766196,C,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,T
9,X,66863125,G,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN,T


There were 21 pathogenic variants to prostate cancer in chromosome X. These variants came from the AR:367 gene.

The AR gene makes the androgen receptor. Androgen receptor binds to androgenic hormones including testosterone. Its functions include being a DNA binding transcription factor that regulates gene expression and also development and maintenance of the male sexual phenotype. According to literature, the androgen receptor is linked with pathogenesis and progression of prostate cancer. An overexpression of AR leads to progression of prostate cancer.

## Identifying subjects carrying pathogenic variants

Now, I will be screening the 1000 genome project to identify subjects carrying prostate cancer pathogenic variants. First, I will shortlist the variants that both appear in the variants identified in the Clinvar database and 1000 genome project. I will be converting vcf file to a zarr file to make handling of large data more manageable.

I will also find the maximum number of alternate alleles across all positions, since this information could be useful when I try to calculate alternate allele frequencies of different alleles. 

In [32]:
import zarr

if "vcf" in dir():
    del vcf

vcf_filename = DATA + "/ALL.chrX.phase3_shapeit2_mvncall_integrated_v1b.20130502.genotypes.vcf.gz"

vcf = allel.read_vcf(str(vcf_filename), ["variants/ID", "variants/numalt"],
                     log=sys.stdout)

[read_vcf] 65536 rows in 3.60s; chunk in 3.60s (18210 rows/s); X :1734128
[read_vcf] 131072 rows in 6.91s; chunk in 3.31s (19777 rows/s); X :3915040
[read_vcf] 196608 rows in 9.86s; chunk in 2.95s (22219 rows/s); X :6474138
[read_vcf] 262144 rows in 12.98s; chunk in 3.12s (21032 rows/s); X :9519142
[read_vcf] 327680 rows in 16.04s; chunk in 3.06s (21388 rows/s); X :12551848
[read_vcf] 393216 rows in 19.11s; chunk in 3.07s (21359 rows/s); X :15550284
[read_vcf] 458752 rows in 22.03s; chunk in 2.92s (22436 rows/s); X :18776831
[read_vcf] 524288 rows in 24.96s; chunk in 2.93s (22385 rows/s); X :21881750
[read_vcf] 589824 rows in 28.30s; chunk in 3.34s (19637 rows/s); X :24574486
[read_vcf] 655360 rows in 31.27s; chunk in 2.98s (22010 rows/s); X :27659891
[read_vcf] 720896 rows in 34.11s; chunk in 2.84s (23101 rows/s); X :30671296
[read_vcf] 786432 rows in 37.00s; chunk in 2.89s (22652 rows/s); X :33194562
[read_vcf] 851968 rows in 40.10s; chunk in 3.10s (21131 rows/s); X :36031556
[read_v

In [35]:
numalt = vcf['variants/numalt']
max_alt = np.max(numalt)
max_alt

9

In [38]:
zarr_file = vcf_filename.replace(".vcf.gz", ".zarr")
if not os.path.exists(zarr_file):
    allel.vcf_to_zarr(str(vcf_filename),
                      str(zarr_file),
                      fields='*', alt_number=max_alt,
                      log=sys.stdout, overwrite=True)

[vcf_to_zarr] 65536 rows in 18.37s; chunk in 18.37s (3567 rows/s); X :1734128
[vcf_to_zarr] 131072 rows in 35.40s; chunk in 17.03s (3847 rows/s); X :3915040
[vcf_to_zarr] 196608 rows in 50.55s; chunk in 15.15s (4326 rows/s); X :6474138
[vcf_to_zarr] 262144 rows in 63.79s; chunk in 13.24s (4949 rows/s); X :9519142
[vcf_to_zarr] 327680 rows in 76.67s; chunk in 12.88s (5086 rows/s); X :12551848
[vcf_to_zarr] 393216 rows in 89.81s; chunk in 13.13s (4990 rows/s); X :15550284
[vcf_to_zarr] 458752 rows in 102.19s; chunk in 12.39s (5290 rows/s); X :18776831
[vcf_to_zarr] 524288 rows in 114.18s; chunk in 11.99s (5466 rows/s); X :21881750
[vcf_to_zarr] 589824 rows in 126.26s; chunk in 12.08s (5426 rows/s); X :24574486
[vcf_to_zarr] 655360 rows in 138.35s; chunk in 12.09s (5418 rows/s); X :27659891
[vcf_to_zarr] 720896 rows in 150.55s; chunk in 12.20s (5370 rows/s); X :30671296
[vcf_to_zarr] 786432 rows in 163.66s; chunk in 13.11s (4999 rows/s); X :33194562
[vcf_to_zarr] 851968 rows in 177.75s; c

Once the VCF file has been converted to Zarr file, I can now create a dataframe extracting the rows that are pathogenic variants of prostate cancer from 1000 genome project Zarr file.

In [42]:
vcfzarr = zarr.open_group(str(zarr_file), mode="r")

pos_idx = np.where(np.isin(vcfzarr["variants/POS"], pathogenic_df.POS))[0]

vdf =  pd.DataFrame(
    dict([(k, vcfzarr["variants/" + k][pos_idx]) 
    for k in ["CHROM", "POS", "REF"]])
)
vdf["ALT"] = vcfzarr["variants/ALT"][pos_idx, 0]

vdf = vdf.merge(pathogenic_df)

vdf

,CHROM,POS,REF,ALT,GENEINFO,CLNDN,CLNSIG,AF_TGP
0,X,66765802,C,T,AR:367,Malignant_tumor_of_prostate|not_provided,Pathogenic,NaN
1,X,66766163,C,G,AR:367,Malignant_tumor_of_prostate,Pathogenic,NaN
2,X,66937326,G,T,AR:367,Malignant_tumor_of_prostate|Prostate_cancer_su...,Pathogenic,0.00053
3,X,66937337,G,A,AR:367,Malignant_tumor_of_prostate|Prostate_cancer,Pathogenic,0.00053


There are a total of 4 variants observed in 1000 genome project. Now that we have an idea of which variants appear in 1000 genome project, we can extract subjects with non-homozygous reference genotype to identify subjects carrying pathogenic variants of prostate cancer. In the genotype array, we would be essentially looking for subjects with genotypes that are not 0/0 for female or 0/. for male.

In [45]:
# reset idx
pos_idx = np.where(np.isin(vcfzarr["variants/POS"], vdf.POS))[0]

gt = allel.GenotypeArray(np.array([vcfzarr["calldata/GT"][i] for i in pos_idx]))

# convert genotype info to pandas dataframe
gt_df = pd.DataFrame(
    gt.to_gt(),
    columns=np.array(vcfzarr["samples"]),
    index="X:" + pd.Series(vcfzarr["variants/POS"]).loc[pos_idx].astype(str)
)

bool_df = (gt_df != b'0/.') & (gt_df != b'0/0')
bool_df.columns.values[bool_df.any(axis=0)]

gt_df[bool_df.columns.values[bool_df.any(axis=0)]]

,HG00321,HG00371,HG01850,HG02031,HG02069,HG02787
X:66765802,b'0/.',b'0/.',b'0/0',b'0/0',b'0/0',b'0/1'
X:66766163,b'0/.',b'0/.',b'0/0',b'1/0',b'0/0',b'0/0'
X:66937326,b'1/.',b'1/.',b'0/0',b'0/0',b'0/0',b'0/0'
X:66937337,b'0/.',b'0/.',b'0/1',b'0/0',b'0/1',b'0/0'


There are 6 subjects that have variants of interest. Now we can create a new dataframe summarizing genotype information from the provided csv file.

In [51]:
df = gt_df.loc[:, bool_df.any(axis=0)].melt(ignore_index=False, var_name="subject_id", value_name="genotype")

# remove reference entries
df = df[(df.genotype != b"0/0") & (df.genotype != b"0/.")].reset_index()

In [52]:
# merge population table to get super population code
meta = pd.read_csv(DATA + "/1kgn_phase3_population.csv")
out = df.merge(meta[["Sample name", "Superpopulation code", "Sex"]], left_on="subject_id", right_on="Sample name")
out = out.rename(columns={
    "index": "variant_of_interest",
    "Sex": "gender",
    "Superpopulation code": "superpopulation_code",
}).drop("Sample name", axis=1)[["subject_id", "gender", "superpopulation_code", "variant_of_interest", "genotype"]]

In [53]:
out

,subject_id,gender,superpopulation_code,variant_of_interest,genotype
0,HG00321,male,EUR,X:66937326,b'1/.'
1,HG00371,male,EUR,X:66937326,b'1/.'
2,HG01850,female,EAS,X:66937337,b'0/1'
3,HG02031,female,EAS,X:66766163,b'1/0'
4,HG02069,female,EAS,X:66937337,b'0/1'
5,HG02787,female,SAS,X:66765802,b'0/1'


From the table above, we see that two male subjects had pathogenic variants of prostate cancer. The two subjects are from European subpopulation.